In [1]:
import os
from dotenv import load_dotenv

import pandas as pd

from h2ogpte import H2OGPTE

In [12]:
df = pd.read_csv('../QandA.csv')
print(df.shape)
df.head()

(25, 6)


,No,Question,Answer,正解ページ,コメント,正解分割ドキュメンント名
0,1,ビールの主な原材料を全て教えてください？,麦芽、ホップ、酵母、水、副原料,4,NaN,1_ビールについて
1,2,ビールの製造工程の3番目の工程は？,煮沸,5,NaN,1_ビールについて
2,3,Craft Beer Marketでは何種類のビールを提供していますか？,9種類,"6,7",NaN,2_ビアスタイルについて
3,4,Craft Beer Marketが提供するビールの中で、アルコール度数の最も高いと考えられ...,SPECIAL BEER（「バーレーワイン」や「ウィートワ\r\nイン」または、その他のスト...,7,NaN,2_ビアスタイルについて
4,5,代表的なエールスタイルのビールを３つ教えてください？,「アメリカン・ペールエール」「イングリッシュ・ペールエー\r\nル」「アンバーエール」「レッ...,6,NaN,2_ビアスタイルについて


In [13]:
ref_docs = df['正解分割ドキュメンント名'].copy()
ref_docs

0                       1_ビールについて
1                       1_ビールについて
2                    2_ビアスタイルについて
3                    2_ビアスタイルについて
4                    2_ビアスタイルについて
5                   3_テイスティングについて
6                   3_テイスティングについて
7                     4_フレーバーについて
8                     4_フレーバーについて
9                  5_グラスの取り扱いについて
10                 5_グラスの取り扱いについて
11                 5_グラスの取り扱いについて
12                 6_ビールの注ぎ方について
13                 6_ビールの注ぎ方について
14    6_ビールの注ぎ方について, 13_APPENDIX
15                 6_ビールの注ぎ方について
16                 7_ビールのガス圧について
17                       12_各種用語名称
18    8_洗浄方法について(ライン:フォーセット:カプラー)
19    8_洗浄方法について(ライン:フォーセット:カプラー)
20              10_器具・用品リスト(カプラー)
21              10_器具・用品リスト(カプラー)
22             11_器具・用品リスト(FAUCET)
23                     13_APPENDIX
24                     13_APPENDIX
Name: 正解分割ドキュメンント名, dtype: object

In [14]:
ref_docs[14] = '13_APPENDIX'

In [23]:
!ls ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt

0_はじめに.txt
10_器具・用品リスト(カプラー).txt
11_器具・用品リスト(FAUCET).txt
12_各種用語名称.txt
13_APPENDIX.txt
1_ビールについて.txt
2_ビアスタイルについて.txt
3_テイスティングについて.txt
4_フレーバーについて.txt
5_グラスの取り扱いについて.txt
6_ビールの注ぎ方について.txt
7_ビールのガス圧について.txt
8_洗浄方法について(ライン:フォーセット:カプラー).txt
9_KEGの取り扱いについて.txt


In [26]:
txt_path = '../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/'

In [38]:
ref_docs_path = txt_path + ref_docs[2] + '.txt'
print(ref_docs_path)

with open(ref_docs_path) as f:
    ref_txt = f.read()
print(ref_txt)

../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/2_ビアスタイルについて.txt

----- Page 6,7 -----

# （2）ビアスタイルについて

## Craft Beer Market’s Beer Menu

Glass : ￥480　Pint : ￥780  
※ Craft Beer Market は上記の2価格で統一する。例外は認められない。

### KANPAI! STANDARD BEER（グラス：中生タンブラー パイント：UK568ml）
日本人が日常的に飲んでるいわゆる”ビール”

### LAGER BEER / PILSNER （グラス：250ml パイント：US473ml）
ラガー酵母(下面発酵酵母)を使用し、低温(10(cid:155)C前後)で長時間かけて発酵させるスタイル。
喉越しが爽やかですっきりとした味わいが特徴。「ピルスナー」「メルツェン」「へレス」などが代表的なラガービールとして挙げられる。

### ALE BEER （グラス：250ml パイント：US473ml）
エール酵母(上面発酵酵母)を使用し、常温(20(cid:155)C前後)゙短期間で発酵させるスタイル。すっきりとした喉越しのラガーに対し、こちらは酵母由来のフルーティな香りやモルトの甘味、ホップの香りをじっくりと味わうビール。 発祥した国や、使われるホップや色の度合いによって様々なスタイルに細分化される。
代表的なエールのスタイル 「アメリカン・ペールエール」「イングリッシュ・ペールエール」「アンバーエール」「レッドエール」「ブラウンエール」「ケルシュ」「アルト」など

### WHITE BEER / WEIZEN （グラス：250ml パイント：US473ml）
小麦を使用したエール酵母(上面発酵酵母)のビール。 色が白く濁ることから白ビールと言われることもある。 柔らかく、フルーティで苦味も少なめな味わいが特徴。 ドイツ発祥の「ヴァイツェン」や、べルギーの「べルジャンホワイト」 が代表的。

### FRUIT/HERB BEER （グラス：250ml パイント：US473ml）
フルーツを使ったビールや、ハーブなどのスパイスを使用したビール。 比較的甘めで苦味が少ないものが多いが、甘味は控えめで香りや酸味

In [43]:
# System Prompts

system_prompt = '''
あなたは質問を注意事項にしたがって回答する優秀なチャットアシスタントです。
必ず日本語で回答します。
長くても70トークン以内で簡潔に回答を作成してください。回答を得た理由を述べる必要はないです。
'''
pre_prompt_query=''
prompt_query=''


# Prompt
message = '''
質問: {}
以下のリファレンスを元に、質問に回答して下さい。
<リファレンス>
{}
</リファレンス>
'''.format(df['Question'][2], ref_txt)

print(message)


質問: Craft Beer Marketでは何種類のビールを提供していますか？
以下のリファレンスを元に、質問に回答して下さい。
<リファレンス>

----- Page 6,7 -----

# （2）ビアスタイルについて

## Craft Beer Market’s Beer Menu

Glass : ￥480　Pint : ￥780  
※ Craft Beer Market は上記の2価格で統一する。例外は認められない。

### KANPAI! STANDARD BEER（グラス：中生タンブラー パイント：UK568ml）
日本人が日常的に飲んでるいわゆる”ビール”

### LAGER BEER / PILSNER （グラス：250ml パイント：US473ml）
ラガー酵母(下面発酵酵母)を使用し、低温(10(cid:155)C前後)で長時間かけて発酵させるスタイル。
喉越しが爽やかですっきりとした味わいが特徴。「ピルスナー」「メルツェン」「へレス」などが代表的なラガービールとして挙げられる。

### ALE BEER （グラス：250ml パイント：US473ml）
エール酵母(上面発酵酵母)を使用し、常温(20(cid:155)C前後)゙短期間で発酵させるスタイル。すっきりとした喉越しのラガーに対し、こちらは酵母由来のフルーティな香りやモルトの甘味、ホップの香りをじっくりと味わうビール。 発祥した国や、使われるホップや色の度合いによって様々なスタイルに細分化される。
代表的なエールのスタイル 「アメリカン・ペールエール」「イングリッシュ・ペールエール」「アンバーエール」「レッドエール」「ブラウンエール」「ケルシュ」「アルト」など

### WHITE BEER / WEIZEN （グラス：250ml パイント：US473ml）
小麦を使用したエール酵母(上面発酵酵母)のビール。 色が白く濁ることから白ビールと言われることもある。 柔らかく、フルーティで苦味も少なめな味わいが特徴。 ドイツ発祥の「ヴァイツェン」や、べルギーの「べルジャンホワイト」 が代表的。

### FRUIT/HERB BEER （グラス：250ml パイント：US473ml）
フルーツを使ったビールや、ハーブなどのスパイスを使用したビール。 比較的甘めで苦味が少ないも

In [48]:
ref_docs

0                       1_ビールについて
1                       1_ビールについて
2                    2_ビアスタイルについて
3                    2_ビアスタイルについて
4                    2_ビアスタイルについて
5                   3_テイスティングについて
6                   3_テイスティングについて
7                     4_フレーバーについて
8                     4_フレーバーについて
9                  5_グラスの取り扱いについて
10                 5_グラスの取り扱いについて
11                 5_グラスの取り扱いについて
12                 6_ビールの注ぎ方について
13                 6_ビールの注ぎ方について
14                     13_APPENDIX
15                 6_ビールの注ぎ方について
16                 7_ビールのガス圧について
17                       12_各種用語名称
18    8_洗浄方法について(ライン:フォーセット:カプラー)
19    8_洗浄方法について(ライン:フォーセット:カプラー)
20              10_器具・用品リスト(カプラー)
21              10_器具・用品リスト(カプラー)
22             11_器具・用品リスト(FAUCET)
23                     13_APPENDIX
24                     13_APPENDIX
Name: 正解分割ドキュメンント名, dtype: object

In [54]:
for i in range(len(ref_docs)):
    ref_docs_path = txt_path + ref_docs[i] + '.txt'
    with open(ref_docs_path) as f:
        ref_txt = f.read()
    #print(ref_txt)
    question = df['Question'][i]
    print(i, ' : ', ref_docs_path, question)

    message = '''
    質問: {}
    以下のリファレンスを元に、質問に回答して下さい。
    <リファレンス>
    {}
    </リファレンス>
    '''.format(question, ref_txt)

    #print(message)

0  :  ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/1_ビールについて.txt ビールの主な原材料を全て教えてください？
1  :  ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/1_ビールについて.txt ビールの製造工程の3番目の工程は？
2  :  ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/2_ビアスタイルについて.txt Craft Beer Marketでは何種類のビールを提供していますか？
3  :  ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/2_ビアスタイルについて.txt Craft Beer Marketが提供するビールの中で、アルコール度数の最も高いと考えられるビールは？
4  :  ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/2_ビアスタイルについて.txt 代表的なエールスタイルのビールを３つ教えてください？
5  :  ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/3_テイスティングについて.txt テイスティングで、口当たりと外観はどちらを先に確認しますか？
6  :  ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/3_テイスティングについて.txt テイスティングの際、どのような条件の時、香りが強くなるか？
7  :  ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/4_フレーバーについて.txt 甘みフレーバーの表現5つを全て答えてください？
8  :  ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/4_フレーバーについて.txt スカンクのような臭いがするオフフレーバーは？
9  :  ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/5_グラスの取り扱いについて.txt グラグラと足付きグラスの見た目の違いを教えてください？
10  :  ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/5_グラスの取り扱いについて.txt リアルエー

In [44]:
!ls -a

.                               .ipynb_checkpoints
..                              Ans_in_prompt.ipynb
.env                            Craft-Beer-Market_run_RAG.ipynb


In [47]:
#!cat .env

In [46]:
load_dotenv(dotenv_path='.env')
heogpteKey = os.getenv('CBM_ref_prompt_NoRAG')
if heogpteKey: print('key is set.')

client = H2OGPTE(
    address='https://h2ogpte.dev.h2o.ai',
    api_key=heogpteKey,
)
client

key is set.


In [72]:
%%time

LlmModel = 'gpt-4o-mini'

chat_session_id = client.create_chat_session_on_default_collection()
print(chat_session_id)


ref_docs_path = txt_path + ref_docs[i] + '.txt'
with open(ref_docs_path) as f:
    ref_txt = f.read()
#print(ref_txt)
question = df['Question'][i]
print(i, ' : ', ref_docs_path, question)

message = '''
質問: {}
以下のリファレンスを元に、質問に回答して下さい。
<リファレンス>
{}
</リファレンス>
'''.format(question, ref_txt)

with client.connect(chat_session_id) as session:
    reply = session.query(
        message = message,
        system_prompt=system_prompt,
        #pre_prompt_query=pre_prompt_query,
        #prompt_query=prompt_query,
        llm=LlmModel,
        rag_config={"rag_type": "llm_only"},)

print('Answer > ', reply.content)

afd4187f-aa53-4b7d-954d-1b1729f093e9
7  :  ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/4_フレーバーについて.txt 甘みフレーバーの表現5つを全て答えてください？
Answer >  甘みフレーバーの表現は以下の5つです。

1. フルーツ感
2. モルト感
3. 蜂蜜感
4. タフィー
5. 糖蜜
CPU times: user 24.5 ms, sys: 5.35 ms, total: 29.8 ms
Wall time: 4.45 s


In [73]:
%%time

LlmModel = 'gpt-4o-mini'    # この部分のみ置き換える

chat_session_id = client.create_chat_session_on_default_collection()
print(chat_session_id)

res = []

for i in range(len(ref_docs)):
    ref_docs_path = txt_path + ref_docs[i] + '.txt'
    with open(ref_docs_path) as f:
        ref_txt = f.read()
    #print(ref_txt)
    question = df['Question'][i]
    print(i, ' : ', ref_docs_path, question)

    message = '''
    質問: {}
    以下のリファレンスを元に、質問に回答して下さい。
    <リファレンス>
    {}
    </リファレンス>
    '''.format(question, ref_txt)

    with client.connect(chat_session_id) as session:
        reply = session.query(
            message = message,
            system_prompt=system_prompt,
            #pre_prompt_query=pre_prompt_query,
            #prompt_query=prompt_query,
            llm=LlmModel,
            rag_config={"rag_type": "llm_only"},   # (Normal) RAG 
        )
        print('Answer > ', reply.content)
        res.append(reply.content)

print(len(res))

mod_name = LlmModel.replace('/', '--')
pd.DataFrame({LlmModel:res}).to_csv('../results/Craft-Beer-Market_noRAG/res__{}.csv'.format(mod_name), index=False)

59994c27-1c6a-44d5-bca1-7f63fa02c57e
0  :  ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/1_ビールについて.txt ビールの主な原材料を全て教えてください？
Answer >  ビールの主な原材料は以下の4つです。  
1. ホップ  
2. 麦芽  
3. 水  
4. 酵母  

副原料として、麦（ライ麦、小麦など）、米、糖類、コーンスターチ、着色料も使用されます。
1  :  ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/1_ビールについて.txt ビールの製造工程の3番目の工程は？
Answer >  ビールの製造工程の3番目は「煮沸」です。この工程では、麦汁を煮沸し、ホップを加えて苦味をつけます。
2  :  ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/2_ビアスタイルについて.txt Craft Beer Marketでは何種類のビールを提供していますか？
Answer >  Craft Beer Marketでは、以下の8種類のビールを提供しています。  
1. KANPAI! STANDARD BEER  
2. LAGER BEER / PILSNER  
3. ALE BEER  
4. WHITE BEER / WEIZEN  
5. FRUIT/HERB BEER  
6. BLACK BEER  
7. IPA – INDIA PALE ALE  
8. REAL ALE  
9. SPECIAL BEER  

合計で9種類のビールがあります。
3  :  ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/2_ビアスタイルについて.txt Craft Beer Marketが提供するビールの中で、アルコール度数の最も高いと考えられるビールは？
Answer >  Craft Beer Marketが提供するビールの中で、アルコール度数が最も高いと考えられるのは「SPECIAL BEER」です。これは長期熟成でハイアルコールのビールに分類されます。
4  :  ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/2_ビアスタイル

In [74]:
%%time

LlmModel = 'meta-llama/Meta-Llama-3.1-8B-Instruct'    # この部分のみ置き換える

chat_session_id = client.create_chat_session_on_default_collection()
print(chat_session_id)

res = []

for i in range(len(ref_docs)):
    ref_docs_path = txt_path + ref_docs[i] + '.txt'
    with open(ref_docs_path) as f:
        ref_txt = f.read()
    #print(ref_txt)
    question = df['Question'][i]
    print(i, ' : ', ref_docs_path, question)

    message = '''
    質問: {}
    以下のリファレンスを元に、質問に回答して下さい。
    <リファレンス>
    {}
    </リファレンス>
    '''.format(question, ref_txt)

    with client.connect(chat_session_id) as session:
        reply = session.query(
            message = message,
            system_prompt=system_prompt,
            #pre_prompt_query=pre_prompt_query,
            #prompt_query=prompt_query,
            llm=LlmModel,
            rag_config={"rag_type": "llm_only"},   # (Normal) RAG 
        )
        print('Answer > ', reply.content)
        res.append(reply.content)

print(len(res))

mod_name = LlmModel.replace('/', '--')
pd.DataFrame({LlmModel:res}).to_csv('../results/Craft-Beer-Market_noRAG/res__{}.csv'.format(mod_name), index=False)

29a39532-c621-4a41-823b-07e0d0ff7549
0  :  ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/1_ビールについて.txt ビールの主な原材料を全て教えてください？
Answer >  ビールの主な原材料は次の4つです。

1. ホップ（セイヨウカラハナソウ）
2. 麦芽（二条大麦）
3. 水
4. 酵母（ラガー酵母やエール酵母）
1  :  ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/1_ビールについて.txt ビールの製造工程の3番目の工程は？
Answer >  ビールの製造工程の3番目の工程は「煮沸」です。
2  :  ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/2_ビアスタイルについて.txt Craft Beer Marketでは何種類のビールを提供していますか？
Answer >  Craft Beer Marketでは、以下の9種類のビールを提供しています。

1. KANPAI! STANDARD BEER
2. LAGER BEER / PILSNER
3. ALE BEER
4. WHITE BEER / WEIZEN
5. FRUIT/HERB BEER
6. BLACK BEER
7. IPA – INDIA PALE ALE
8. REAL ALE
9. SPECIAL BEER
3  :  ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/2_ビアスタイルについて.txt Craft Beer Marketが提供するビールの中で、アルコール度数の最も高いと考えられるビールは？
Answer >  Craft Beer Marketが提供するビールの中で、アルコール度数の最も高いと考えられるビールは「SPECIAL BEER」です。
4  :  ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/2_ビアスタイルについて.txt 代表的なエールスタイルのビールを３つ教えてください？
Answer >  代表的なエールスタイルのビールは以下の3つです。

1. アメリカン・ペールエール
2. イングリッシュ・ペールエール
3. アンバ

In [75]:
%%time

LlmModel = 'Qwen/Qwen2.5-72B-Instruct'    # この部分のみ置き換える

chat_session_id = client.create_chat_session_on_default_collection()
print(chat_session_id)

res = []

for i in range(len(ref_docs)):
    ref_docs_path = txt_path + ref_docs[i] + '.txt'
    with open(ref_docs_path) as f:
        ref_txt = f.read()
    #print(ref_txt)
    question = df['Question'][i]
    print(i, ' : ', ref_docs_path, question)

    message = '''
    質問: {}
    以下のリファレンスを元に、質問に回答して下さい。
    <リファレンス>
    {}
    </リファレンス>
    '''.format(question, ref_txt)

    with client.connect(chat_session_id) as session:
        reply = session.query(
            message = message,
            system_prompt=system_prompt,
            #pre_prompt_query=pre_prompt_query,
            #prompt_query=prompt_query,
            llm=LlmModel,
            rag_config={"rag_type": "llm_only"},   # (Normal) RAG 
        )
        print('Answer > ', reply.content)
        res.append(reply.content)

print(len(res))

mod_name = LlmModel.replace('/', '--')
pd.DataFrame({LlmModel:res}).to_csv('../results/Craft-Beer-Market_noRAG/res__{}.csv'.format(mod_name), index=False)

cb53f2c9-7126-4270-9b33-063f36e57c2c
0  :  ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/1_ビールについて.txt ビールの主な原材料を全て教えてください？
Answer >  まず、質問は「ビールの主な原材料を全て教えてください？」です。リファレンスを見ると、ビールの主な原材料としてホップ、麦芽、水、酵母が挙げられています。副原料も法律で定められているものが記載されていますが、主な原材料は上記の4つです。それぞれの役割や特徴も説明されていますが、質問は原材料の名称を知りたいだけなので、これらを簡潔に列挙します。
</think>

ビールの主な原材料は、ホップ、麦芽、水、酵母です。
1  :  ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/1_ビールについて.txt ビールの製造工程の3番目の工程は？
Answer >  Alright, let's tackle this question. The user is asking about the third step in the beer manufacturing process. I remember from the reference that the process is outlined on Page 5. 

First, I need to list out the steps in order. The first step is糖化 (sugarization), where they make the malt extract. The second isろ過 (filtration), where they separate the liquid from the solids. The third step is煮沸 (boiling), where they add hops to get the bitterness. 

So, the third step is煮沸. I should make sure to mention that this is where the hops are added to create the苦味 (bitterness)

In [76]:
%%time

LlmModel = 'gpt-4o'    # この部分のみ置き換える

chat_session_id = client.create_chat_session_on_default_collection()
print(chat_session_id)

res = []

for i in range(len(ref_docs)):
    ref_docs_path = txt_path + ref_docs[i] + '.txt'
    with open(ref_docs_path) as f:
        ref_txt = f.read()
    #print(ref_txt)
    question = df['Question'][i]
    print(i, ' : ', ref_docs_path, question)

    message = '''
    質問: {}
    以下のリファレンスを元に、質問に回答して下さい。
    <リファレンス>
    {}
    </リファレンス>
    '''.format(question, ref_txt)

    with client.connect(chat_session_id) as session:
        reply = session.query(
            message = message,
            system_prompt=system_prompt,
            #pre_prompt_query=pre_prompt_query,
            #prompt_query=prompt_query,
            llm=LlmModel,
            rag_config={"rag_type": "llm_only"},   # (Normal) RAG 
        )
        print('Answer > ', reply.content)
        res.append(reply.content)

print(len(res))

mod_name = LlmModel.replace('/', '--')
pd.DataFrame({LlmModel:res}).to_csv('../results/Craft-Beer-Market_noRAG/res__{}.csv'.format(mod_name), index=False)

91249d68-36fc-4e1c-ba9e-30ac35ee8c5b
0  :  ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/1_ビールについて.txt ビールの主な原材料を全て教えてください？
Answer >  ビールの主な原材料は以下の通りです：

1. ホップ
2. 麦芽
3. 水
4. 酵母
5. 副原料（麦、米、糖類、コーンスターチ、こうりやん、ばれいしょ、着色料）
1  :  ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/1_ビールについて.txt ビールの製造工程の3番目の工程は？
Answer >  ビールの製造工程の3番目の工程は「煮沸」です。
2  :  ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/2_ビアスタイルについて.txt Craft Beer Marketでは何種類のビールを提供していますか？
Answer >  Craft Beer Marketでは、9種類のビールを提供しています。
3  :  ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/2_ビアスタイルについて.txt Craft Beer Marketが提供するビールの中で、アルコール度数の最も高いと考えられるビールは？
Answer >  Craft Beer Marketが提供するビールの中で、アルコール度数が最も高いと考えられるビールは「SPECIAL BEER」です。
4  :  ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/2_ビアスタイルについて.txt 代表的なエールスタイルのビールを３つ教えてください？
Answer >  代表的なエールスタイルのビールは以下の3つです：

1. アメリカン・ペールエール
2. イングリッシュ・ペールエール
3. アンバーエール
5  :  ../data/CRAFT-BEER-MARKET_品質管理マニュアル_txt/3_テイスティングについて.txt テイスティングで、口当たりと外観はどちらを先に確認しますか？
Answer >  テイスティングでは、外観を先に確認します。
6  :  ../data/CRAFT-BEER-MARKET